In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import mnist
import mlflow
import mlflow.tensorflow
from concurrent.futures import ThreadPoolExecutor

# Enable auto-logging to MLflow to capture TensorFlow training runs
mlflow.tensorflow.autolog()

In [2]:
# Load and preprocess the MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0  # Normalize pixel values to [0, 1]
x_train = x_train.reshape(-1, 28, 28, 1)
x_test = x_test.reshape(-1, 28, 28, 1)

In [3]:
# Define multiple configurations for hyperparameters
configurations = [
    {"optimizer": "adam", "epochs": 2, "filters": 32, "num_layers": 2},
    {"optimizer": "sgd", "epochs": 2, "filters": 64, "num_layers": 3},
    {"optimizer": "rmsprop", "epochs": 2, "filters": 16, "num_layers": 1}
]


In [4]:
# Function to create the model with variable filters and layers
def create_model(filters, num_layers):
    model = models.Sequential()
    model.add(layers.Conv2D(filters, (3, 3), activation='relu', input_shape=(28, 28, 1)))
    model.add(layers.MaxPooling2D((2, 2)))
    
    # Add additional layers based on the num_layers parameter
    for _ in range(num_layers - 1):
        model.add(layers.Conv2D(filters * 2, (3, 3), activation='relu'))
        model.add(layers.MaxPooling2D((2, 2)))
    
    model.add(layers.Flatten())
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(10, activation='softmax'))
    return model

In [5]:
# Define a function to train a model with a given configuration
def train_model(config):
    optimizer = config["optimizer"]
    epochs = config["epochs"]
    filters = config["filters"]
    num_layers = config["num_layers"]
    
    # Create the model based on filters and layers configuration
    model = create_model(filters, num_layers)

    # Start MLflow experiment
    mlflow.set_experiment("mnist_classification_Pejman")

    with mlflow.start_run():
        # Log parameters
        mlflow.log_param("optimizer", optimizer)
        mlflow.log_param("epochs", epochs)
        mlflow.log_param("filters", filters)
        mlflow.log_param("num_layers", num_layers)
        mlflow.log_param("Author", "Pejman")

        # Compile the model with the specific optimizer
        model.compile(optimizer=optimizer,
                      loss='sparse_categorical_crossentropy',
                      metrics=['accuracy'])

        # Train the model
        model.fit(x_train, y_train, epochs=epochs, validation_data=(x_test, y_test))

        # Evaluate the model
        test_loss, test_acc = model.evaluate(x_test, y_test, verbose=2)

        # Log metrics
        mlflow.log_metric("test_loss", test_loss)
        mlflow.log_metric("test_accuracy", test_acc)

        # Log the model with a name based on the configuration
        model_name = f"mnist_model_{optimizer}_{epochs}_epochs_{filters}_filters_{num_layers}_layers"
        mlflow.tensorflow.log_model(model, artifact_path=model_name)

        print(f"Finished training with optimizer={optimizer}, epochs={epochs}, filters={filters}, num_layers={num_layers}")

In [6]:
for config in configurations:
    train_model(config)

2024/09/16 09:33:30 INFO mlflow.tracking.fluent: Experiment with name 'mnist_classification_Pejman' does not exist. Creating a new experiment.


Epoch 1/2


2024-09-16 09:33:30.735230: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


1875/1875 [==============================] - 15s 8ms/step - loss: 0.1493 - accuracy: 0.9544 - val_loss: 0.0529 - val_accuracy: 0.9845
Epoch 2/2
1/1 [==============================] - 0s 51ms/step


INFO:tensorflow:Assets written to: /var/folders/jr/24zqc_wx7m7bq37h2_n26wbw0000gn/T/tmpazbycyr9/model/data/model/assets


...

313/313 - 0s - loss: 0.0607 - accuracy: 0.9798 - 367ms/epoch - 1ms/step


2024/09/16 09:35:47 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


INFO:tensorflow:Assets written to: /var/folders/jr/24zqc_wx7m7bq37h2_n26wbw0000gn/T/tmpefxj8aqx/model/data/model/assets


INFO:tensorflow:Assets written to: /var/folders/jr/24zqc_wx7m7bq37h2_n26wbw0000gn/T/tmpefxj8aqx/model/data/model/assets
2024/09/16 09:35:49 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Finished training with optimizer=rmsprop, epochs=2, filters=16, num_layers=1
